In [1]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
import pandas as pd
import numpy as np
import os
import joblib
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import Descriptors

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr

DESC_NAMES = [name for name, _ in Descriptors.descList]

def compute_rdkit_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [np.nan] * len(DESC_NAMES)
    return [func(mol) for _, func in Descriptors.descList]


def compute_descriptor_df(df, smiles_col="smiles"):
    desc_values = df[smiles_col].apply(compute_rdkit_descriptors)
    desc_df = pd.DataFrame(desc_values.tolist(), columns=DESC_NAMES)
    return desc_df
def select_correlated_descriptors(X, y, threshold=0.10, min_samples=30):
    """
    Select descriptors with |Pearson r| >= threshold using pairwise complete data.
    """
    selected = []
    correlations = {}

    y = np.asarray(y)

    for col in X.columns:
        x = X[col].values

        # Mask valid pairs
        mask = np.isfinite(x) & np.isfinite(y)

        if mask.sum() < min_samples:
            continue

        x_valid = x[mask]
        y_valid = y[mask]

        # Skip zero-variance descriptors
        if np.std(x_valid) == 0:
            continue

        r, _ = pearsonr(x_valid, y_valid)

        if np.isfinite(r) and abs(r) >= threshold:
            selected.append(col)
            correlations[col] = r

    print(f"✅ Selected {len(selected)} descriptors with |r| ≥ {threshold}")
    return selected, pd.Series(correlations).sort_values(key=np.abs, ascending=False)



df_train = pd.read_csv("./NIHDataset/train_df.csv")
df_val   = pd.read_csv("./NIHDataset/val_df.csv")
df_test  = pd.read_csv("./NIHDataset/test_df.csv")

for df in [df_train, df_val, df_test]:
    df.rename(columns={"SMILES_CANON": "smiles", "logLD50": "LD50"}, inplace=True)


In [2]:
print("🔹 Computing RDKit descriptors...")
X_train_desc = compute_descriptor_df(df_train)
X_val_desc   = compute_descriptor_df(df_val)
X_test_desc  = compute_descriptor_df(df_test)

y_train = df_train["LD50"].values
y_val   = df_val["LD50"].values
y_test  = df_test["LD50"].values


selected_desc, corr_series = select_correlated_descriptors(
    X_train_desc,
    y_train,
    threshold=0.05
)


X_train_sel = X_train_desc[selected_desc]
X_val_sel   = X_val_desc[selected_desc]
X_test_sel  = X_test_desc[selected_desc]
imputer = KNNImputer(n_neighbors=5, weights="distance")

X_train_imp = imputer.fit_transform(X_train_sel)
X_val_imp   = imputer.transform(X_val_sel)
X_test_imp  = imputer.transform(X_test_sel)
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train_imp)
X_val   = scaler.transform(X_val_imp)
X_test  = scaler.transform(X_test_imp)

print("✅ Final feature shapes:")
print("Train:", X_train.shape)
print("Val  :", X_val.shape)
print("Test :", X_test.shape)


🔹 Computing RDKit descriptors...
✅ Selected 92 descriptors with |r| ≥ 0.05
✅ Final feature shapes:
Train: (7118, 92)
Val  : (890, 92)
Test : (890, 92)


In [3]:
gbr = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    random_state=42
)

hgb = HistGradientBoostingRegressor(
    max_iter=500,
    learning_rate=0.05,
    max_depth=6,
    l2_regularization=1e-3,
    random_state=42
)
gbr.fit(X_train, y_train)
hgb.fit(X_train, y_train)

pred_val_gbr = gbr.predict(X_val)
pred_val_hgb = hgb.predict(X_val)

weights = np.linspace(0, 1, 101)
mae_scores = []

for w in weights:
    blended = w * pred_val_gbr + (1 - w) * pred_val_hgb
    mae = mean_absolute_error(y_val, blended)
    mae_scores.append(mae)

best_w = weights[np.argmin(mae_scores)]
print(f"\n✅ Optimal ensemble weight (GBR): {best_w:.2f}")
print(f"✅ Optimal ensemble weight (HGB): {1 - best_w:.2f}")
def ensemble_predict(X):
    return (
        best_w * gbr.predict(X) +
        (1 - best_w) * hgb.predict(X)
    )
def evaluate(y_true, y_pred, label=""):
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2   = r2_score(y_true, y_pred)
    pcc, _ = pearsonr(y_true, y_pred)

    print(f"\n📊 {label}")
    print(f"MAE  : {mae:.4f}")
    print(f"RMSE : {rmse:.4f}")
    print(f"R²   : {r2:.4f}")
    print(f"PCC  : {pcc:.4f}")

    return mae, rmse, r2, pcc

val_preds = ensemble_predict(X_val)
evaluate(y_val, val_preds, "Internal Validation (df_val)")

test_preds = ensemble_predict(X_test)
evaluate(y_test, test_preds, "External Validation (df_test)")



✅ Optimal ensemble weight (GBR): 0.00
✅ Optimal ensemble weight (HGB): 1.00

📊 Internal Validation (df_val)
MAE  : 0.4140
RMSE : 0.5754
R²   : 0.5031
PCC  : 0.7105

📊 External Validation (df_test)
MAE  : 0.4168
RMSE : 0.5718
R²   : 0.4806
PCC  : 0.6937


(0.4168248525193359, 0.5718498229480379, 0.4806360552424248, 0.69366920712522)

In [4]:
def fold_error_percentages(y_true, y_pred):
    abs_err = np.abs(y_pred - y_true)

    within_2fold = np.mean(abs_err <= np.log10(2)) * 100
    within_3fold = np.mean(abs_err <= np.log10(3)) * 100

    return within_2fold, within_3fold

val_preds = ensemble_predict(X_val)

w2, w3 = fold_error_percentages(y_val, val_preds)

print("\n📊 Internal Validation (df_val)")
print(f"Within 2-fold (%) : {w2:.2f}")
print(f"Within 3-fold (%) : {w3:.2f}")


test_preds = ensemble_predict(X_test)

w2, w3 = fold_error_percentages(y_test, test_preds)

print("\n📊 External Validation (df_test)")
print(f"Within 2-fold (%) : {w2:.2f}")
print(f"Within 3-fold (%) : {w3:.2f}")



📊 Internal Validation (df_val)
Within 2-fold (%) : 50.67
Within 3-fold (%) : 68.99

📊 External Validation (df_test)
Within 2-fold (%) : 48.76
Within 3-fold (%) : 67.19
